___

<a href='https://mainacademy.ua/'> <img src='https://mainacademy.ua/wp-content/uploads/2019/03/logo-main.png' alt = 'Хмм, щось з інтернетом'/></a>
___

# Module 4: Work with data professionally!

## Lab work 4



#### Мета: 

* навчитися обробляти дані
* "парсити" сайти

### Завдання 1:

#### Ознайомитися із API Національного банку 
https://old.bank.gov.ua/control/uk/publish/article?art_id=38441973&cat_id=38459171#exchange



1. Підключитися до API.


In [1]:
import requests
r = requests.get('https://bank.gov.ua/NBUStatService/v1/statdirectory/exchange?json').json()

2. Отримати курс долара за останній рік.

In [2]:
import requests
from datetime import datetime, timedelta

BASE = {}

def con_to_api(date):
    param = {'date' : date.strftime('%Y%m%d')}
    r = requests.get('https://bank.gov.ua/NBUStatService/v1/statdirectory/exchange?json', params=param)
    if r.status_code == 200:
        return r.json()
    return None
date = datetime.today()
for i in range(365):
    date = date - timedelta(days=1)
    r = con_to_api(date)
    for el in r:
        if el['cc'] == 'USD':
            BASE[date] = el["rate"]
            
print(BASE)

{datetime.datetime(2021, 4, 18, 10, 24, 41, 808081): 27.9592, datetime.datetime(2021, 4, 17, 10, 24, 41, 808081): 27.9592, datetime.datetime(2021, 4, 16, 10, 24, 41, 808081): 27.9592, datetime.datetime(2021, 4, 15, 10, 24, 41, 808081): 27.9765, datetime.datetime(2021, 4, 14, 10, 24, 41, 808081): 28.0156, datetime.datetime(2021, 4, 13, 10, 24, 41, 808081): 27.9335, datetime.datetime(2021, 4, 12, 10, 24, 41, 808081): 27.9094, datetime.datetime(2021, 4, 11, 10, 24, 41, 808081): 27.9768, datetime.datetime(2021, 4, 10, 10, 24, 41, 808081): 27.9768, datetime.datetime(2021, 4, 9, 10, 24, 41, 808081): 27.9768, datetime.datetime(2021, 4, 8, 10, 24, 41, 808081): 27.8923, datetime.datetime(2021, 4, 7, 10, 24, 41, 808081): 27.8384, datetime.datetime(2021, 4, 6, 10, 24, 41, 808081): 27.939, datetime.datetime(2021, 4, 5, 10, 24, 41, 808081): 27.9555, datetime.datetime(2021, 4, 4, 10, 24, 41, 808081): 27.8324, datetime.datetime(2021, 4, 3, 10, 24, 41, 808081): 27.8324, datetime.datetime(2021, 4, 2, 1

3. Вивести середнє значення та відхилення курсу за кожний місяць.

In [3]:
from statistics import stdev, mean
# Переводимо словник до списку кортежів
dict_to_list = [i for i in BASE.items()]

month_dict = {}
list_month = []

# Отримуємо перший місяць
first_month = dict_to_list[0][0].month

for i in dict_to_list:
    m = i[0].month
    list_month.append(i[1])
    if first_month != m:
        # Наповнюэмо словник де ключем є місяць, а значенням кортеж зі значеннямми: середнього та відхилення
        month_dict[first_month] = (mean(list_month), stdev(list_month))
        list_month = []
        first_month = m
            

month_dict  

{4: (27.919642105263158, 0.062217631618461014),
 3: (27.7970064516129, 0.11694869797047422),
 2: (27.89404642857143, 0.12105005540146793),
 1: (28.2224064516129, 0.12343287767172319),
 12: (28.17596129032258, 0.2548431291041863),
 11: (28.30916, 0.14990632155242592),
 10: (28.32045483870968, 0.05153356730461981),
 9: (27.949283333333334, 0.24966056002077097),
 8: (27.528367741935483, 0.15101206880488424),
 7: (27.280609677419356, 0.37368535191596586),
 6: (26.714466666666667, 0.08127598505786574),
 5: (26.81644193548387, 0.13668813841294206)}

In [142]:
import pickle 
with open(r'C:\Users\Алекс\Documents\github\DS-6\Lab work 4\districts.pickle', 'rb') as f:     #pickle працює в бінарному режимі 
    res = pickle.load(f) 
    
res

,total,transport,infrastructure,rest,security,eko
Голосіїв,10.0,10.0,10.0,10.0,10.0,10.0
Оболонь,9.7,9.7,9.9,9.8,9.7,9.6
Мишоловка,9.6,8.0,10.0,10.0,10.0,10.0
Теремки-2,9.0,9.0,9.0,9.0,9.0,9.0
Китаїв,8.7,8.8,6.0,9.3,9.5,10.0
...,...,...,...,...,...,...
Монтажник,1.0,1.0,1.0,1.0,1.0,1.0
Жовтневе,1.0,1.0,1.0,1.0,1.0,1.0
Петрівка,1.0,1.0,1.0,1.0,1.0,1.0
Перемога,1.0,1.0,1.0,1.0,1.0,1.0


In [193]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

HEADERS = {
            'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Mobile Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
            'Accept-Language': 'ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3',
            'Accept-Encoding': 'gzip, deflate, br',
            'DNT': '1',
            'Connection': 'keep-alive',
        }

metro_district = ['Печерский', 'Шевченковский']
district_dict = {'Печерский': 2.87, 'Шевченковский': 1.74, 'Голосеевский': 1.53, 'Подольский': 1.29, 'Оболонский': 1.22, 'Соломенский': 1.19, 'Днепровский': 1.19, 'Святошинский': 1.18, 'Дарницкий': 1.12, 'Деснянский': 1.00}

district_list = []
area_list = []
rooms_list = []
floor_list = []
near_metro_list = []
gk_list = []
adress_list = []
price_list = []




for page in range (1, 188):
    SITE = f'https://rieltor.ua/ru/flats-rent/?page={page}'

    r = requests.get(SITE, headers=HEADERS)

    if r.status_code == 200:
        soup = BeautifulSoup(r.text, 'lxml')


        for el in soup.find_all('div', class_='catalog-item'):
            adress = el.find_all('a')[1].text.split()
            str_adres = ' '.join(adress)
            try:
                adress_list.append(str_adres)# адрес (район)
            except ValueError as e:
                adress_list.append(None)
            try:
                district = el.find_all('a')[1].text.split()[-2]
                number_distr = district_dict.get(district)
                district_list.append(number_distr)# number район
            except IndexError as e:
                district_list.append(None)
            metro = el.find_all('a')[2].text.split()
            #print(metro)
            # метро
            try:
                if '(M)' in metro:
                    #print(1, 'metro')
                    near_metro_list.append(1)
                elif district in metro_district:
                    #print(1, 'metro po rayonu')
                    near_metro_list.append(1)
                
#                 elif el.find('p', class_='catalog-item_info-item-row catalog-item_info-description') != None:
#                     if 'метро' in el.find('p', class_='catalog-item_info-item-row catalog-item_info-description').text:
#                         near_metro_list.append(2)
                    
                    
                else:
                    near_metro_list.append(3)
                    
            except AttributeError as e:
                near_metro_list.append(None)


            price = el.find('strong', class_='catalog-item__price').text # цена
            if '$/мес' in price:
                price_list.append(int(price.rstrip('$/мес').replace(' ', '')) * 27.5)
            else:
                price_list.append(int(price.rstrip('грн/мес').replace(' ', ''))) 
            try:
                rooms = int(el.find('div', class_='catalog-item_info-item-row').text.split()[0])
                rooms_list.append(rooms)#комнаты
            except ValueError as e:
                rooms_list.append(None)

            try:
                floor = int(el.find('div', class_='catalog-item_info-item-row').text.split()[2])
                floor_list.append(floor)   # этаж
            except ValueError as e:
                floor_list.append(None)
            try:    
                area = float(el.find('div', class_='catalog-item_info-item-row').text.split()[-6])
                area_list.append(area)# площадь
            except ValueError as e:
                area_list.append(None)
            try:
                gk = el.find('p', class_='catalog-item_info-item-row catalog-item_info-description').text
                if 'ЖК' in gk:
                    #print(int(1))
                    gk_list.append(1)
                elif 'комплекс%' in gk:
                    gk_list.append(1)
                else:
                    #print(int(0))
                    gk_list.append(0)
            except AttributeError as e:
                    gk = None
                    #print(int(0))
                    gk_list.append(0)


    df = pd.DataFrame({'district': district_list, 'metro': near_metro_list, 'area': area_list, 'rooms': rooms_list, 'floor': floor_list, 'gk': gk_list, 'price': price_list, 'adress': adress_list})

            
df       
#         l = el.find('a')
#         link = l.get('href')
#         r1 = requests.get(link, headers = HEADERS)
#         soup1 = BeautifulSoup(r1.text, 'lxml')
#         bd_list1.append(
#                 (int(soup1.find('span', class_='sku').text[5:]),
#                 soup1.find_all('h2')[1].text,
#                 float(soup1.find('span', class_='regular-price').text.rstrip(' грн\n').replace(' ', '')),
#                 soup1.find_all('p')[6].text)
#             )


#bd_list1


# conn = sqlite3.connect('data_shop.sqlite3')
# curs = conn.cursor()

# script = """CREATE TABLE IF NOT EXISTS dobriva ( 
#                 dobriva_id INT PRIMARY KEY,
#                 name TEXT NOT NULL,
#                 price DECIMAL(8,2) NOT NULL,
#                 weight TEXT
#                 );
# """

# curs.executescript(script)
# conn.close()

# for product_info in bd_list:
#     conn = sqlite3.connect('data_shop.sqlite3')
#     curs = conn.cursor()

#     s = "INSERT INTO dobriva VALUES (?, ?, ?, ?)"
#     try:
#         curs.execute(s, product_info)
#         conn.commit()
#     except Exception as e:
#         print(e)

#     conn.close()

,district,metro,area,rooms,floor,gk,price,adress
0,2.87,1,70.0,2.0,18.0,1,33000.0,"Евгения Коновальца ул. (Щорса), 44а, Печерский..."
1,1.53,3,75.0,3.0,4.0,0,46750.0,"Деловая ул., 13, Голосеевский р-н"
2,2.87,1,96.0,2.0,10.0,0,35750.0,"Михаила Омеляновича-Павленка ул. (Суворова), П..."
3,2.87,1,120.0,3.0,5.0,1,95975.0,"Драгомирова ул., 20, Печерский р-н"
4,1.74,1,50.0,2.0,11.0,1,27500.0,"Златоустовская ул., 30, Шевченковский р-н"
...,...,...,...,...,...,...,...,...
3735,1.18,3,25.0,1.0,2.0,0,4700.0,"Семьи Сосниных ул., Святошинский р-н"
3736,1.19,3,33.0,1.0,1.0,0,7500.0,"Соборности просп. (Воссоединения), Днепровский..."
3737,1.18,1,55.0,1.0,1.0,0,3500.0,"Тулузы ул., Святошинский р-н"
3738,1.22,3,70.0,3.0,7.0,0,3000.0,"Полярная ул., Оболонский р-н"


4. Дану інформацію записати у файл за допомогою pickle.

In [4]:
import pickle

with open(r'month_mean_dev.pickle', 'wb') as f:
    pickle.dump(month_dict, f)

### Завдання 2:

Потрібно проаналізувати всі товари на сайті: 
https://smallpacking.agrosem.ua/shop/

Створіть базу даних, що містить таблицю із такими стовбцями: 
    - артикул товару
    - найменування
    - ціна 
    - вага

### Завдання 3: 

Ознайомтеся із роботою SQLite та відповідним модулем у Python.
Завантажте базу даних для виконання лабораторної роботи.
Підключіться до завантаженої бази SQLite.

1. Виведіть інформацію про дану базу.

In [15]:
conn=sqlite3.connect('tysql.sqlite')
curs=conn.cursor()
tables=[]
script="""SELECT name 
          FROM sqlite_master 
          WHERE type='table';"""
res = conn.execute(script)
for name in res:
    tables.append(name[0])
for i in range(len(tables)):    
    print(tables[i])
    name=tables[i]
    s=f"pragma table_info({name});"
    curs.execute(s)
    while True:
        next_row = curs.fetchone()
        if next_row:
            print(next_row)
        else:
            break

Customers
(0, 'cust_id', 'char(10)', 1, None, 1)
(1, 'cust_name', 'char(50)', 1, None, 0)
(2, 'cust_address', 'char(50)', 0, None, 0)
(3, 'cust_city', 'char(50)', 0, None, 0)
(4, 'cust_state', 'char(5)', 0, None, 0)
(5, 'cust_zip', 'char(10)', 0, None, 0)
(6, 'cust_country', 'char(50)', 0, None, 0)
(7, 'cust_contact', 'char(50)', 0, None, 0)
(8, 'cust_email', 'char(255)', 0, None, 0)
OrderItems
(0, 'order_num', 'int', 1, None, 1)
(1, 'order_item', 'int', 1, None, 2)
(2, 'prod_id', 'char(10)', 1, None, 0)
(3, 'quantity', 'int', 1, None, 0)
(4, 'item_price', 'decimal(8,2)', 1, None, 0)
Orders
(0, 'order_num', 'int', 1, None, 1)
(1, 'order_date', 'datetime', 1, None, 0)
(2, 'cust_id', 'char(10)', 1, None, 0)
Products
(0, 'prod_id', 'char(10)', 1, None, 1)
(1, 'vend_id', 'char(10)', 1, None, 0)
(2, 'prod_name', 'char(255)', 1, None, 0)
(3, 'prod_price', 'decimal(8,2)', 1, None, 0)
(4, 'prod_desc', 'text', 0, None, 0)
Vendors
(0, 'vend_id', 'char(10)', 1, None, 1)
(1, 'vend_name', 'char(50)

2. Виведіть перелік всіх таблиць.

In [16]:
script="""SELECT name 
          FROM sqlite_master 
          WHERE type='table';"""
res = conn.execute(script)
for name in res:
    print(name[0])

Customers
OrderItems
Orders
Products
Vendors


3. Список всіх cust_id з таблиці Customers table.

In [17]:
script="""SELECT cust_id 
          FROM Customers 
          ;"""
res = conn.execute(script)
print(res.fetchall())
for name in res:
    print(name[0])

[('1000000001',), ('1000000002',), ('1000000003',), ('1000000004',), ('1000000005',)]


4. Всю таблицю Customers table

In [18]:
curs.execute("""SELECT * 
                FROM Customers;""")
while True:
    next_row = curs.fetchone()
    if next_row:
        print(next_row)
    else:
        break

('1000000001', 'Village Toys', '200 Maple Lane', 'Detroit', 'MI', '44444', 'USA', 'John Smith', 'sales@villagetoys.com')
('1000000002', 'Kids Place', '333 South Lake Drive', 'Columbus', 'OH', '43333', 'USA', 'Michelle Green', None)
('1000000003', 'Fun4All', '1 Sunny Place', 'Muncie', 'IN', '42222', 'USA', 'Jim Jones', 'jjones@fun4all.com')
('1000000004', 'Fun4All', '829 Riverside Drive', 'Phoenix', 'AZ', '88888', 'USA', 'Denise L. Stephens', 'dstephens@fun4all.com')
('1000000005', 'The Toy Store', '4545 53rd Street', 'Chicago', 'IL', '54545', 'USA', 'Kim Howard', None)


5. Список клієнтів (cust_names) відсортованих від Z до A

In [20]:
curs.execute("""SELECT cust_name 
                FROM Customers
                ORDER BY cust_name DESC;""")
while True:
    next_row = curs.fetchone()
    if next_row:
        print(next_row)
    else:
        break

('Village Toys',)
('The Toy Store',)
('Kids Place',)
('Fun4All',)
('Fun4All',)


6. Таблицю клієнтів та замовлень (cust_id і order_num) . Відсортуйте по клієнту і потім по даті замовлення

In [21]:
curs.execute("""SELECT cust_id, order_num 
                FROM Orders
                ORDER BY cust_id, order_date;""")
while True:
    next_row = curs.fetchone()
    if next_row:
        print(next_row)
    else:
        break

('1000000001', 20009)
('1000000001', 20005)
('1000000003', 20006)
('1000000004', 20007)
('1000000005', 20008)


7. Таблицю (на основі Items) з кількість та вартістю товару. Відсортуйте в порядку спадання по кількості та вартості

In [22]:
curs.execute("""SELECT quantity, item_price
                FROM OrderItems
                ORDER BY quantity DESC, item_price DESC;""")
while True:
    next_row = curs.fetchone()
    if next_row:
        print(next_row)
    else:
        break

(250, 2.49)
(250, 2.49)
(250, 2.49)
(100, 10.99)
(100, 5.49)
(100, 2.99)
(100, 2.99)
(100, 2.99)
(50, 11.49)
(50, 4.49)
(20, 5.99)
(10, 11.99)
(10, 8.99)
(10, 3.49)
(10, 3.49)
(10, 3.49)
(5, 11.99)
(5, 4.99)


8. Товар (з таблиці Products), ціна якого становить 9.49

In [23]:
curs.execute("""SELECT prod_name, prod_price
                FROM Products
                WHERE prod_price=9.49;""")
while True:
    next_row = curs.fetchone()
    if next_row:
        print(next_row)
    else:
        break

('King doll', 9.49)
('Queen doll', 9.49)


9. Виведіть назву товару та ціну, яка лежить в діапазоні від  3 до 6. Відсортуйте результат в по ціні в порядку зростання

In [24]:
curs.execute("""SELECT prod_name, prod_price
                FROM Products
                WHERE prod_price BETWEEN 3 AND 6
                GROUP BY prod_price;""")
while True:
    next_row = curs.fetchone()
    if next_row:
        print(next_row)
    else:
        break

('Fish bean bag toy', 3.49)
('Raggedy Ann', 4.99)
('8 inch teddy bear', 5.99)


10. Кількість товару, що було продано

In [25]:
curs.execute("""SELECT SUM(quantity)
                FROM OrderItems;""")
while True:
    next_row = curs.fetchone()
    if next_row:
        print(next_row)
    else:
        break

(1430,)


11. Кількість найменувань товару, ціна якого більша за 4

In [26]:
curs.execute("""SELECT COUNT(prod_price)
                FROM Products
                WHERE prod_price>4;""")
while True:
    next_row = curs.fetchone()
    if next_row:
        print(next_row)
    else:
        break

(6,)


12. Розробіть алгоритм для виведення 3 найдорожчих товарів в базі

In [27]:
curs.execute("""SELECT prod_name, prod_price 
                FROM Products
                GROUP BY prod_price
                HAVING prod_price IN (SELECT DISTINCT prod_price 
                                     FROM Products
                                     ORDER BY prod_price DESC LIMIT 3)
                             
                ;""")
while True:
    next_row = curs.fetchone()
    if next_row:
        print(next_row)
    else:
        break


('12 inch teddy bear', 8.99)
('King doll', 9.49)
('18 inch teddy bear', 11.99)


13. Підрахуйте кількість замовлень для кожного клієнта використовуючи підзапити

In [28]:
curs.execute("""SELECT cust_id, COUNT(order_num)
                FROM (SELECT Customers.cust_id, Orders.order_num
                      FROM Customers
                      LEFT JOIN Orders ON Customers.cust_id=Orders.cust_id) as t1
                GROUP BY cust_id;""")

while True:
    next_row = curs.fetchone()
    if next_row:
        print(next_row)
    else:
        break

('1000000001', 2)
('1000000002', 0)
('1000000003', 1)
('1000000004', 1)
('1000000005', 1)


14. Виведіть список клієнтів та їх замовлення (використайте JOIN)

In [29]:
curs.execute("""SELECT cust_name, order_num, order_date
                FROM Orders
                JOIN Customers ON Orders.cust_id=Customers.cust_id;""")
while True:
    next_row = curs.fetchone()
    if next_row:
        print(next_row)
    else:
        break

('Village Toys', 20005, '2012-05-01')
('Fun4All', 20006, '2012-01-12')
('Fun4All', 20007, '2012-01-30')
('The Toy Store', 20008, '2012-02-03')
('Village Toys', 20009, '2012-02-08')


15. Зробіть два запити: список товарів, ціна яких менша 5 та список товарів, ціна який більша рівна 5. Використайте UNION для об'єднання цих запитів

In [30]:
curs.execute("""SELECT prod_name, prod_price
                FROM Products
                WHERE prod_price<5
                UNION
                SELECT prod_name, prod_price
                FROM Products
                WHERE prod_price>=5
                ORDER BY prod_price
                ;""")
while True:
    next_row = curs.fetchone()
    if next_row:
        print(next_row)
    else:
        break

('Bird bean bag toy', 3.49)
('Fish bean bag toy', 3.49)
('Rabbit bean bag toy', 3.49)
('Raggedy Ann', 4.99)
('8 inch teddy bear', 5.99)
('12 inch teddy bear', 8.99)
('King doll', 9.49)
('Queen doll', 9.49)
('18 inch teddy bear', 11.99)


### Завдання 4: 

Зареєструйтеся на сайті  http://www.sql-ex.ru/

Виконайте завдання із блоку SELECT; з 1 по 30 завдання

Код запитів вставляйте сюди


In [ ]:
#1
SELECT model, speed,  hd
FROM PC
WHERE price < 500
#2
SELECT DISTINCT maker
FROM Product 
WHERE type = 'Printer'
#3
SELECT model, ram, screen
FROM laptop
WHERE price > 1000
#4
SELECT *
FROM printer
WHERE color = 'y'
#5
SELECT model, speed, hd
FROM pc
WHERE (cd = '12x' OR cd = '24x') AND price < 600
#6
SELECT DISTINCT Product.maker, Laptop.speed 
FROM Product INNER JOIN 
 Laptop ON Laptop.model = Product.model
WHERE Laptop.hd >= 10
#7
select s.model, s.price 
from 
(SELECT model, price FROM laptop
UNION 
SELECT model, price FROM pc
UNION
SELECT model, price FROM printer) as s
JOIN product on s.model=product.model
WHERE maker ='B'
#8
SELECT maker FROM product
WHERE type = 'pc' AND maker NOT IN
(SELECT maker FROM product WHERE type = 'laptop')
GROUP BY maker
#9
SELECT DISTINCT maker FROM product
WHERE model in (SELECT model FROM pc WHERE speed >= 450)
#10
SELECT model, price  
FROM Printer  
WHERE price = (SELECT MAX(price) 
               FROM Printer
               )
#11
SELECT AVG(speed) AS Avg_speed
FROM pc
#12
SELECT AVG(speed) AS Avg_speed
FROM LAPTOP
WHERE price > 1000
#13
SELECT AVG(pc.speed) AS Avg_speed
FROM pc
INNER JOIN product ON product.model = pc.model
WHERE product.maker = 'A'
#14
SELECT DISTINCT classes.class, ships.name, classes.country FROM Ships
INNER JOIN classes ON classes.class = ships.class
WHERE classes.numGuns >= 10
#15
SELECT DISTINCT hd FROM pc
GROUP BY hd
HAVING COUNT(hd) > = 2
